# Code to create agents and teams

This notebook shows the code flow to create agents and teams using flo-ai

In [1]:
from flo_ai import FloSupervisor, FloAgent, FloSession, FloTeam, FloLinear
from langchain_openai import ChatOpenAI
# TODO router config needs fixing
from flo_ai.yaml.flo_team_builder import RouterConfig
from langchain_community.tools.tavily_search.tool import TavilySearchResults
from dotenv import load_dotenv

load_dotenv()

True

Code to create a simple tea, with 2 agents, each agent having one tool of itself

In [2]:
llm = ChatOpenAI(temperature=0, model_name='gpt-4o')
session = FloSession(llm).register_tool(
    name="TavilySearchResults", 
    tool=TavilySearchResults()
)

researcher = FloAgent.Builder(session, "Reasercher", 
                              "Do a research on the internet and find articles of relevent to the topic asked by the user", [TavilySearchResults()]).build()
blogger = FloAgent.Builder(session, "BlogWriter", 
                           "Able to write a blog using information provided", [TavilySearchResults()]).build()

marketing_team = FloTeam.Builder(session, "Marketing", [researcher, blogger]).build()
r1 = FloSupervisor.Builder(session, "Head-of-Marketing", marketing_team).build()
flo_team_1 = r1.build_routed_team()


In [3]:
def draw_to_file(graph, filename: str, xray=False):
    from PIL import Image as PILImage
    import io
    byte_image = graph.draw(xray)
    with io.BytesIO(byte_image) as image_io:
        image = PILImage.open(image_io)
        image.save(filename)

In [4]:
draw_to_file(flo_team_1, "dk.png", True)

A second team created for doing editorial suggestions

In [5]:
chief_editorial = FloAgent.Builder(session, "Senior-Editor", 
                              "Have a look at the article created and give editorial suggestions", [TavilySearchResults()]).build()

# TODO needs builder
edit_team = FloTeam.Builder(session, "Editorial-Team", [chief_editorial]).build()
r2 = FloSupervisor.Builder(session, "Editor-Team-Routing", edit_team).build()

editorial_team = r2.build_routed_team()
draw_to_file(editorial_team, "editoial.png", True)

In [6]:
journal_company = FloTeam.Builder(session, "Newspaper", [flo_team_1, editorial_team])
r3 = FloLinear.Builder(session,
                        journal_company, 
                        RouterConfig(**{"name" :"Serial", "kind": "linear"})).build()

master_team = r3.build_routed_team()

In [7]:
draw_to_file(master_team, "master.png")

In [8]:
master_team.invoke("Write an article about CR7")



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Cristiano Ronaldo 2023 news'}`


[{'url': 'https://abcnews.go.com/Sports/cristiano-ronaldo-end-2023-worlds-top-goalscorer/story?id=106012559', 'content': "By ESPN NEWS SERVICES. December 30, 2023, 2:39 PM. Cristiano Ronaldo is set to end the year as the world's top goalscorer after scoring his 54th goal of 2023 in Al Nassr 's 4-1 victory at Al ..."}, {'url': 'https://www.cnn.com/2023/12/12/sport/cristiano-ronaldo-50-goals-2023-spt-intl/index.html', 'content': 'CNN values your feedback\nCristiano Ronaldo scores 50th goal of 2023 as potential Lionel Messi clash is announced\nCristiano Ronaldo scored his 50th goal of the calendar year on Monday as Al Nassr advanced to the semifinals of the Saudi King Cup with a 5-2 victory against rival Al Shabab.\n Lionel Messi says he is focused on playing at next year’s Copa América but doesn’t rule out appearing at 2026 World Cup\nLast week, the 38-year-ol

BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'messages[2].name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'messages[2].name', 'code': 'invalid_value'}}